In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2500)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [12]:
os.getcwd()

'/home/model-vision-dev/workspace/kkj/helloworld/NLP/NLG'

In [13]:
import glob
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
files = glob.glob(os.getcwd() + "/webnlg-dataset/release_v3.0/en/train/**/*.xml", recursive=True)
triple_re=re.compile('(\d)triples')
bdata_dct={}
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('webNLG')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df=pd.DataFrame(mdata_dct)
df.to_csv('webNLG2020_train.csv')

In [26]:
df.loc[0, 'input_text']

'Aarhus_Airport | cityServed | "Aarhus, Denmark"'

In [27]:
df.loc[0, 'target_text']

'The Aarhus is the airport of Aarhus, Denmark.'

In [24]:
df.loc[30000, 'input_text']

'Allen_Forrest | birthYear | 1981 && Allen_Forrest | birthPlace | "Fort Campbell, KY, raised in Dothan, AL" && Allen_Forrest | background | "solo_singer" && Allen_Forrest | genre | Pop_music && Allen_Forrest | birthPlace | Fort_Campbell'

In [25]:
df.loc[30000, 'target_text']

'Allen Forrest is a solo pop music singer who was born in 1981 in Fort Campbell, Kentucky but was raised in Dothan, Alabama.'

In [28]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [29]:
train_df=pd.read_csv('webNLG2020_train.csv', index_col=[0])
train_df=train_df.iloc[  :35000,:]
train_df=train_df.sample(frac = 1)
batch_size=8
num_of_batches=len(train_df)/batch_size

In [30]:
train_df

,prefix,input_text,target_text
30617,webNLG,Alessio_Romagnoli | currentclub | A.C._Milan &...,"Alessio Romagnoli, who has represented Italy a..."
31463,webNLG,"Anaheim,_California | isPartOf | Orange_County...","Anaheim is in Orange County, Greater Los Angel..."
16424,webNLG,United_Petrotrin_F.C. | ground | Palo_Seco_Vel...,Akeem Adams plays for the T&TEC Sports Club an...
1330,webNLG,United_Kingdom | capital | London,The capital city of the United Kingdom is London.
24656,webNLG,107_Camilla | discoverer | N._R._Pogson && N._...,"N.R. Pogson, born in Nottingham, was the disco..."
19253,webNLG,A-Rosa_Luna | shipDisplacement | 1850.0 (tonne...,The cruise ship A-Rosa Luna weighs 1850 tonnes...
12813,webNLG,Adam_Koc | battle | Polish–Soviet_War && Polis...,"A commander at the Polish-Soviet War, which Ad..."
11742,webNLG,Bionico | dishVariation | Cottage_cheese && Bi...,Bionico is a dish served for dessert that can ...
23160,webNLG,Andrew_Rayel | associatedBand/associatedMusica...,"Trance musician Andrew Rayel, who began his ca..."
23783,webNLG,Canada | anthem | O_Canada && Canada | leader ...,O Canada is the national anthem of Canada and ...


In [34]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [35]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [36]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [37]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [39]:
num_of_epochs = 10

In [42]:
#Sets the module in training mode
model.train()

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in range(int(num_of_batches)):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'WebNLG: '+row['input_text']+'</s>' 
      labels = row['target_text']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
    inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    if i%10 ==0:      
      loss_per_10_steps.append(loss_num)
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:186: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:557: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


Epoch: 1 , Running loss: 0.5035280405810901
Running epoch: 2


Epoch: 2 , Running loss: 0.3812277759364673
Running epoch: 3


Epoch: 3 , Running loss: 0.33595852660281317
Running epoch: 4


...

Epoch: 6 , Running loss: 0.26267075616121294
Running epoch: 7


Epoch: 7 , Running loss: 0.24567667559215
Running epoch: 8


Epoch: 8 , Running loss: 0.23040025485583715
Running epoch: 9


Epoch: 9 , Running loss: 0.21628123998471668
Running epoch: 10


Epoch: 10 , Running loss: 0.20402244181292398


In [43]:
torch.save(model.state_dict(),'pytoch_model.bin')

* config 파일 필요..?

In [55]:
import wget

In [56]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json')

't5-base-config.json'

In [50]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [63]:
model =T5ForConditionalGeneration.from_pretrained(os.getcwd(), 
                                                return_dict=True)
def generate(text,model,tokenizer):
   model.eval()
   input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), 
                               return_tensors="pt")  
   outputs = model.generate(input_ids)
   return tokenizer.decode(outputs[0])

In [108]:
generate('porsche | money | rich', model, tokenizer)

'<pad> The money used in Paorsche is rich.</s>'